# Trajectory 사람 이동 예측
trajectory 데이터에서 총 180명 정도의 데이터가 존재 <br>
데이터 출처 : https://www.microsoft.com/en-us/download/details.aspx?id=52367&from=http%3A%2F%2Fresearch.microsoft.com%2Fen-us%2Fdownloads%2Fb16d359d-d164-469e-9fd4-daa38f2b2e13%2F<br>
개인 별로 어떻게 움직일지를 lstm, gru (파라미터가 좀 작을 때 사용하는 모델),transfomer 모형을 활용해서 학습해볼 예정 <br>
## 표본 선정 기준<br>
1. 수집 일 수가 많을 것 <br>
2. 일정하게 데이터가 존재할 것 <br>
결과적으로 총 6명을 학습 할 예정이다. <br>

In [1]:
import os
import pandas as pd
list_ = os.listdir('sorted_data/128/Trajectory/')
sorted_41 = []
for i in range(len(list_)):
    file_ = open('sorted_data/128/Trajectory/' + list_[i], 'r') 
    file = file_.readlines()
    file = file[6:]
    for j in range(len(file)):
        tmp = file[j].split(',')
        sorted_41.append([float(tmp[0]), float(tmp[1]), tmp[-2] + ' ' +tmp[-1][:-1]])

df = pd.DataFrame(sorted_41)
df[2] = pd.to_datetime(df[2])
df.sort_values(by = [2],inplace=True)
df = df.reset_index()
df = df[[0,1,2]]

In [2]:
len(df)

1208500

# 데이터 분리하기
1. 데이터가 너무 많다는 점
2. 어느 곳에 방문할지에 대한 정보가 좀 더 중요하다는 점
3. 사람이 걸을 수 있는 정도가 한정되어 있다는 점 <br>
세가지를 고려할 때 모두를 학습시키기 보다는 일부분을 쪼개서 학습시키는 것이 합리적

In [3]:
a = []
for i in range(1208499):
    if i % 30 == 0:
        diff = df[2].iloc[i + 1] - df[2].iloc[i]
        a.append(diff)

In [4]:
result = a[0]
for i in range(1,len(a)):
    result += a[i]

In [5]:
result / len(a)

Timedelta('0 days 00:01:27.645094826')

# 30분 단위로 쪼개서 이 사람의 미래 예측하는 형태
    평균이 1분 27초 정도되니 30정도 곱해서 30분 단위로 예측해볼 것임

In [6]:
df_diff_30 = []
for i in range(0,len(df),30):
    df_diff_30.append(df.iloc[i])

In [7]:
len(df_diff_30)

40284

In [8]:
df_diff_30_pd = pd.DataFrame(df_diff_30)

In [9]:
df_diff_30_pd[0], df_diff_30_pd[1], df_diff_30_pd[2]

(0          40.075850
 30         40.076483
 60         40.051017
 90         40.046900
 120        40.040267
              ...    
 1208370    39.987238
 1208400    39.986165
 1208430    39.985208
 1208460    39.979902
 1208490    39.976603
 Name: 0, Length: 40284, dtype: float64,
 0          116.327817
 30         116.321833
 60         116.323883
 90         116.320683
 120        116.325833
               ...    
 1208370    116.402507
 1208400    116.364760
 1208430    116.330910
 1208460    116.327333
 1208490    116.331755
 Name: 1, Length: 40284, dtype: float64,
 0         2007-04-14 00:56:28
 30        2007-04-14 01:10:40
 60        2007-04-14 01:35:35
 90        2007-04-14 01:48:46
 120       2007-04-14 01:59:23
                   ...        
 1208370   2011-03-10 07:48:47
 1208400   2011-03-10 07:51:51
 1208430   2011-03-10 07:54:49
 1208460   2011-03-10 07:57:23
 1208490   2011-03-10 07:59:42
 Name: 2, Length: 40284, dtype: datetime64[ns])

# 30분 단위로 쪼갠다!

In [10]:
df_128_30 = pd.DataFrame(df_diff_30)

In [11]:
x, y = df_128_30[0], df_128_30[1]

# 모델 학습 데이터 준비
    x, y 차원을 맞추는 것이 어려운 점이었음
    gru 모델 2개를 활용해서 한 차원 당 1개의 모델을 사용하도록 하였음.
    

In [12]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)-n_steps):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-2], sequences[end_ix, -2:]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# x, y
# define input sequence
in_seq1 = x.values
in_seq2 = y.values
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

OSError: [WinError 1455] 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다. Error loading "C:\Users\SungJin\anaconda3\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
df = split_sequences(dataset, 5)

In [ ]:
df

# GRU 모델 학습

In [169]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 50 # number of hidden states
        self.n_layers = 10 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.GRU(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [170]:
n_features = 1 # this is number of parallel inputs
n_timesteps = 5 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net_x = MV_LSTM(n_features, n_timesteps)
mv_net_y = MV_LSTM(n_features, n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer_x = torch.optim.Adam(mv_net_x.parameters(), lr=1e-1)
optimizer_y = torch.optim.Adam(mv_net_y.parameters(), lr=1e-1)

train_episodes = 500
batch_size = 20

(40279, 5, 2) (40279, 2)


In [132]:
type(X) 
# 여기서 타입 에러가 발생했음 # float 타입으로 결과를 만들어야 했는데
# 위에서 string 타입으로 데이터 프레임을 만들어내서 문제가 발생

numpy.ndarray

In [133]:
X

array([[[ 40.07585   , 116.32781667],
        [ 40.07648333, 116.32183333],
        [ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333]],

       [[ 40.07648333, 116.32183333],
        [ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333],
        [ 40.03626667, 116.31031667]],

       [[ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333],
        [ 40.03626667, 116.31031667],
        [ 40.01473333, 116.31171667]],

       ...,

       [[ 39.9705499 , 116.4605233 ],
        [ 39.9647216 , 116.464055  ],
        [ 39.98799   , 116.4282633 ],
        [ 39.9872383 , 116.4025066 ],
        [ 39.986165  , 116.3647599 ]],

       [[ 39.9647216 , 116.464055  ],
        [ 39.98799   , 116.4282633 ],
        [ 39.9872383 , 116.4025066 ],
        [ 39.986165  , 116.3647599 ],
        [ 39.9852083 , 116.3309099 ]],

       [[ 39.98799   , 116.

# 여기서도 어려웠던 점
    인풋 사이즈를 맞춰줘야 하는데 그 자체가 코드를 모두 이해하여야 했기에 엄청난 어려움이 있었다.
    ★시계열 데이터이기 때문에 마지막 배치의 경우에 문제가 존재했음 -> 이 부분을 처리하는 것이 중요!

In [142]:
y

array([[ 40.03626667, 116.31031667],
       [ 40.01473333, 116.31171667],
       [ 39.99198333, 116.30961667],
       ...,
       [ 39.9852083 , 116.3309099 ],
       [ 39.9799016 , 116.3273333 ],
       [ 39.9766033 , 116.3317549 ]])

In [145]:
y.shape

(40279, 2)

In [171]:
mv_net_x.train()
mv_net_y.train()

for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]

        # x, y 트레인셋
        x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32)
        y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32)

        if x_batch.shape == (batch_size,n_timesteps):
            x_batch = x_batch.reshape(batch_size,n_timesteps,1) # 3 채널로 맞춰줘야 함
            y_batch = y_batch.reshape(batch_size,n_timesteps,1)

            mv_net_x.init_hidden(x_batch.size(0))
            mv_net_y.init_hidden(y_batch.size(0))

            # x,y의 예측 값
            output_x = mv_net_x(x_batch)
            output_y = mv_net_y(y_batch)

            # x,y의 정답
            target_x_batch = torch.tensor(target[:, 0],dtype=torch.float32)
            target_y_batch = torch.tensor(target[:, 1],dtype=torch.float32)


        #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
        #    lstm_out.contiguous().view(x_batch.size(0),-1)

            # loss function이 두개의 합이 최소가 되는 쪽으로 loss를 구함
            loss = criterion(output_x.view(-1), target_x_batch) + criterion(output_y.view(-1), target_y_batch)
            loss.backward()

            optimizer_x.step()        
            optimizer_x.zero_grad() # x가 0으로 흐르도록 함

            optimizer_y.step()
            optimizer_y.zero_grad() # y가 0으로 흐르도록 함
    print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  245.24386596679688
step :  1 loss :  232.52220153808594
step :  2 loss :  229.64219665527344
step :  3 loss :  228.66375732421875
step :  4 loss :  228.3067169189453
step :  5 loss :  228.17318725585938
step :  6 loss :  228.28575134277344
step :  7 loss :  228.1910858154297
step :  8 loss :  228.129638671875
step :  9 loss :  228.1063232421875
step :  10 loss :  228.09756469726562
step :  11 loss :  228.09390258789062
step :  12 loss :  228.09263610839844
step :  13 loss :  228.09226989746094
step :  14 loss :  228.09222412109375
step :  15 loss :  228.09222412109375
step :  16 loss :  228.09207153320312
step :  17 loss :  228.0917510986328
step :  18 loss :  228.09226989746094
step :  19 loss :  228.09207153320312
step :  20 loss :  228.0921630859375
step :  21 loss :  228.09194946289062
step :  22 loss :  228.09207153320312


KeyboardInterrupt: 